# [LVV-1615 (v1.0)] - M2 Integration with SAL

[LVV-1615 (v1.0)]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T1615

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import rubin_jupyter_utils.lab.notebook as nb
nb.utils.get_node()

/tmp/ipykernel_21772/1665379685.py:2: DeprecationWarning: Call to deprecated function (or staticmethod) get_node. (Please use lsst.rsp.get_node())
  nb.utils.get_node()


'yagan07'

In [3]:
from lsst.sitcom import vandv

exec_info = vandv.ExecutionInfo()
print(exec_info)


Executed by isotuela on 2022-06-15T15:15:36.897.
  Running in yagan07 at summit



In [4]:
import asyncio
import os
import yaml

import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from astropy import time 
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
from datetime import datetime, timedelta

from lsst_efd_client import EfdClient
from lsst.ts import utils, salobj
from lsst.ts.cRIOpy import M1M3FATable
from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control import RotType

import lsst.sitcom.vandv as vandv

In [5]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])
print(os.environ["LSST_DDS_DOMAIN_ID"])

file:///opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-3.0.0/lib/python3.8/config/ospl-shmem.xml
summit


KeyError: 'LSST_DDS_DOMAIN_ID'

In [6]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [7]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [8]:
domain = salobj.Domain()

In [9]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

In [10]:
await mtcs.start_task

[None, None, None, None, None, None, None, None, None, None]

In [11]:
if exec_info.loc == "summit":
    client = EfdClient("summit_efd")
elif location == "tucson":
    client = EfdClient("tucson_teststand_efd")
else:
    raise ValueError(
        "Location does not match any valid options {summit|tucson}"
    )

In [29]:
index = 16151296  # Test Case + Test Execution

start_time = datetime.now()
script = salobj.Controller("Script", index=index)

In [30]:
script.log.info("LVV-T1615 - LVV-E1296 - Start")

## M2 DDS Startup Procedure

---
## Connect to https://ls.st/hexrot-vm01

Make sure that you have an IPA account and that your username is part of the saluser group.

Access https://ls.st/hexrot-vm01 using a browser (Firefox recommended).

Log in with your account and open a terminal.

---
## Start MTM2 EUI

Using the terminal, navigate to the /rubin/mtm2/build folder and execute runM2Cntlr.
Use the example code below to check if there is another session before starting your own.
If there is another session running, it is recommended that you contact the user and ask them to close it.

---
## Use the chronograf to verify data is being published to the EFD.

---
## Transition the MTM2 CSC into DISABLED state either through LOVE or through Jupyter Notebook. 

--- 
Remote Control

In [13]:
m2_commandable_by_dds = mtcs.rem.mtm2.evt_commandableByDDS.get()
print(m2_commandable_by_dds)

private_revCode: 1f79b729, private_sndStamp: 1654631347.9880934, private_rcvStamp: 1655223124.9253323, private_seqNum: 17, private_identity: MTM2, private_origin: 25375, state: True, priority: 0


---
Transition the MTM2 CSC into ENABLED state through LOVE.

In [15]:
import numpy as np
axial_forces = np.zeros(72)
axial_forces[0] = 20 # in N - This is Actuator B1 

await mtcs.rem.mtm2.cmd_applyForces.set_start(axial=axial_forces)

In [20]:
np.degrees(6e-6)

0.00034377467707849394

In [ ]:
start = "2022"

---
resetForceOffsets

In [21]:
await mtcs.rem.mtm2.cmd_resetForceOffsets.start()

---
Query data to the EFD

In [33]:
start = "2022-06-14T15:35:00"
start = time.Time(start, format="isot", scale="utc")

end = time.Time(utils.current_tai(), format="unix", scale="tai")
end.format = "isot"

print(start, end)

2022-06-14T15:35:00.000 2022-06-14T18:46:53.274


In [34]:
df = await client.select_time_series(
    "lsst.sal.MTM2.axialForce", 
    fields="*", 
    start=start.utc, 
    end=end.utc,
)

df

,applied0,applied1,applied10,applied11,applied12,applied13,applied14,applied15,applied16,applied17,...,measured8,measured9,private_efdStamp,private_identity,private_kafkaStamp,private_origin,private_rcvStamp,private_revCode,private_seqNum,private_sndStamp
2022-06-14 16:06:13.385000+00:00,0,0,0,0,0,0,0,0,0,0,...,184.69,189.34,1.655223e+09,MTM2,1.655223e+09,25375,1.655223e+09,1af5d334,11023,1.655223e+09
2022-06-14 16:06:13.548000+00:00,0,0,0,0,0,0,0,0,0,0,...,184.60,189.33,1.655223e+09,MTM2,1.655223e+09,25375,1.655223e+09,1af5d334,11024,1.655223e+09
2022-06-14 16:06:14.396000+00:00,0,0,0,0,0,0,0,0,0,0,...,184.70,189.36,1.655223e+09,MTM2,1.655223e+09,25375,1.655223e+09,1af5d334,11025,1.655223e+09
2022-06-14 16:06:14.562000+00:00,0,0,0,0,0,0,0,0,0,0,...,184.75,189.30,1.655223e+09,MTM2,1.655223e+09,25375,1.655223e+09,1af5d334,11026,1.655223e+09
2022-06-14 16:06:14.724000+00:00,0,0,0,0,0,0,0,0,0,0,...,184.84,189.46,1.655223e+09,MTM2,1.655223e+09,25375,1.655223e+09,1af5d334,11027,1.655223e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-14 18:45:39.695000+00:00,0,0,0,0,0,0,0,0,0,0,...,184.66,189.10,1.655232e+09,MTM2,1.655232e+09,25375,1.655232e+09,1af5d334,12581,1.655232e+09
2022-06-14 18:45:43.706000+00:00,0,0,0,0,0,0,0,0,0,0,...,184.74,189.18,1.655232e+09,MTM2,1.655232e+09,25375,1.655232e+09,1af5d334,12582,1.655232e+09
2022-06-14 18:45:50.721000+00:00,0,0,0,0,0,0,0,0,0,0,...,184.54,188.93,1.655232e+09,MTM2,1.655232e+09,25375,1.655232e+09,1af5d334,12583,1.655232e+09
2022-06-14 18:45:58.741000+00:00,0,0,0,0,0,0,0,0,0,0,...,184.72,189.08,1.655232e+09,MTM2,1.655232e+09,25375,1.655232e+09,1af5d334,12584,1.655232e+09


In [36]:
np.degrees(100e-6) * 3600

20.626480624709636

NameError: name 'x' is not defined

In [38]:
await mtcs.rem.mtm2.cmd_clearErrors.set_start()

In [40]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtm2"])

In [41]:
await mtcs.rem.mtm2.cmd_positionMirror.set_start(x=0)

In [42]:
await mtcs.rem.mtm2.cmd_positionMirror.set_start(y=0)

In [43]:
await mtcs.rem.mtm2.cmd_positionMirror.set_start(yRot=0)

In [44]:
await mtcs.rem.mtm2.cmd_positionMirror.set_start(z=5)

In [16]:
print(mtcs.rem.mtm2.evt_m2AssemblyInPosition.get())

private_revCode: 56870c9f, private_sndStamp: 1655237829.3627574, private_rcvStamp: 1655306152.4709733, private_seqNum: 36158, private_identity: MTM2, private_origin: 25375, inPosition: False, priority: 0


---
Query events to the EFD 

In [42]:
# Define the start and end time of the 
start = "2022-06-14T16:30:00"
start = time.Time(start, format="isot", scale="utc")

end = "2022-06-14T17:30:00"
end = time.Time(end, format="isot", scale="utc")

print(start, end)

2022-06-14T16:30:00.000 2022-06-14T17:30:00.000


---
M2 Assembly detailed state

Verify the MTM2_logevent_detailedState event is published to the EFD.

In [27]:
await client.select_time_series(
    "lsst.sal.MTM2.logevent_detailedState", 
    fields="*", 
    start=start, 
    end=end,
)


""


---
M2 Assembly Controller State

Verify the MTM2_logevent_controllerState event is published to the EFD.

In [28]:
await client.select_time_series(
    "lsst.sal.MTM2.logevent_controllerState", 
    fields="*", 
    start=start, 
    end=end,
)


""


---
M2 Assembly inPosition

Verify that MTM2_logevent_m2AssemblyInPosition event is published to the EFD.

In [43]:
await client.select_time_series(
    "lsst.sal.MTM2.logevent_m2AssemblyInPosition", 
    fields="*", 
    start=start, 
    end=end,
)


,inPosition,priority,private_efdStamp,private_identity,private_kafkaStamp,private_origin,private_rcvStamp,private_revCode,private_seqNum,private_sndStamp
2022-06-14 16:30:01.325000+00:00,True,0,1.655224e+09,MTM2,1.655224e+09,25375,1.655224e+09,56870c9f,31937,1.655224e+09
2022-06-14 16:30:01.426000+00:00,False,0,1.655224e+09,MTM2,1.655224e+09,25375,1.655224e+09,56870c9f,31938,1.655224e+09
2022-06-14 16:30:01.528000+00:00,True,0,1.655224e+09,MTM2,1.655224e+09,25375,1.655224e+09,56870c9f,31939,1.655224e+09
2022-06-14 16:30:01.631000+00:00,False,0,1.655224e+09,MTM2,1.655224e+09,25375,1.655224e+09,56870c9f,31940,1.655224e+09
2022-06-14 16:30:01.732000+00:00,True,0,1.655224e+09,MTM2,1.655224e+09,25375,1.655224e+09,56870c9f,31941,1.655224e+09
...,...,...,...,...,...,...,...,...,...,...
2022-06-14 17:16:17.301000+00:00,False,0,1.655227e+09,MTM2,1.655227e+09,25375,1.655227e+09,56870c9f,34970,1.655227e+09
2022-06-14 17:16:17.402000+00:00,True,0,1.655227e+09,MTM2,1.655227e+09,25375,1.655227e+09,56870c9f,34971,1.655227e+09
2022-06-14 17:16:17.503000+00:00,False,0,1.655227e+09,MTM2,1.655227e+09,25375,1.655227e+09,56870c9f,34972,1.655227e+09
2022-06-14 17:19:14.252000+00:00,True,0,1.655227e+09,MTM2,1.655227e+09,25375,1.655227e+09,56870c9f,34973,1.655227e+09


---
Temperature Events

Verify the MTM2_logevent_CellTemperatureHiWarning and MTM2_logevent_temperatureOffset to the EFD

In [30]:
await client.select_time_series(
    "lsst.sal.MTM2.logevent_CellTemperatureHiWarning", 
    fields="*", 
    start=start, 
    end=end,
)


""


In [31]:
await client.select_time_series(
    "lsst.sal.MTM2.logevent_temperatureOffset", 
    fields="*", 
    start=start, 
    end=end,
)


""


In [ ]:
---
Interlock
Verify the MTM2_logevent_interlock is published to the EFD.

In [38]:
await client.select_time_series(
    "lsst.sal.MTM2.logevent_interlock", 
    fields="*", 
    start=start, 
    end=end,
)


""


---
Jump to Step 37
Hardpoint List

Verify the MTM2_logevent_hardpointList is published to the EFD.

In [40]:
await client.select_time_series(
    "lsst.sal.MTM2.logevent_hardpointList", 
    fields="*", 
    start=start, 
    end=end,
)


""


---
Inclination Telemetry Source

Verify the MTM2_logevent_inclinationTelemetrySource is published in the EFD.


In [41]:
await client.select_time_series(
    "lsst.sal.MTM2.logevent_inclinationTelemetrySource", 
    fields="*", 
    start=start, 
    end=end,
)


""


Force Balance System Status

Verify the MTM2_logevent_forceBalanceSystemStatus is being published to the EFD.

In [44]:
await client.select_time_series(
    "lsst.sal.MTM2.logevent_forceBalanceSystemStatus", 
    fields="*", 
    start=start, 
    end=end,
)


""
